**# IMPORT & EXPLORATION**

# IMPORT & EXPLORATION

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
current_directory = os.getcwd()
print(current_directory)

/content


In [ ]:
# File CSV in DataFrame pandas
from google.colab import files
uploaded = files.upload()
skills = pd.read_csv('./SKILLS_V2.csv', sep = ';')

KeyboardInterrupt: ignored

In [ ]:
skills.head()

SOCIAL_PROFILE_ID  PREDICTED_GENDER_ID  \
0          200445650                  1.0   
1          180037292                  1.0   
2          180136727                  1.0   
3          180136727                  1.0   
4          180568556                  NaN   

                                           RAW_TITLE               SKILL  \
0  Transportation Networks • Urban Mobility • Dat...      Urban Planning   
1    AI Architect. AI Ethics Expert. Data scientist.        Microsoft C#   
2  Big Data Science Architect and Data in the Clo...  Big Data Analytics   
3  Big Data Science Architect and Data in the Clo...    Data Integration   
4  Data Engineer-Data Integration  presso Sistemi...   Microsoft Windows   

  IS_PARSED_FROM_PROFILE_SUMMARY IS_INFERRED_FROM_EXPERIENCE  \
0                         FALSCH                      FALSCH   
1                         FALSCH                      FALSCH   
2                           WAHR                      FALSCH   
3                         FALSCH                      FALSCH   
4                           WAHR                      FALSCH   

  IS_PARSED_FROM_SKILLS_SECTION  
0                          WAHR  
1                          WAHR  
2                        FALSCH  
3                          WAHR  
4                        FALSCH

In [ ]:
skills.columns

Index(['SOCIAL_PROFILE_ID', 'PREDICTED_GENDER_ID', 'RAW_TITLE', 'SKILL',
       'IS_PARSED_FROM_PROFILE_SUMMARY', 'IS_INFERRED_FROM_EXPERIENCE',
       'IS_PARSED_FROM_SKILLS_SECTION'],
      dtype='object')

In [ ]:
#total count of skill records
skills.shape[0]

139229

In [ ]:
#check number of single social profiles aka users
skills.groupby('SOCIAL_PROFILE_ID').size().sort_values(ascending = False)

SOCIAL_PROFILE_ID
251448314    69
349081215    68
148591219    68
235474552    64
248918847    63
             ..
395096289     1
564195279     1
394974413     1
394494326     1
623732754     1
Length: 15484, dtype: int64

In [ ]:
#checking nr of single profiles
skills['SOCIAL_PROFILE_ID'].nunique()

15484

In [ ]:
#unique delivers distinct results in an array, whereas nunique tallies the number of occurrences
skills['SOCIAL_PROFILE_ID'].unique()

array([200445650, 180037292, 180136727, ..., 477052773,  97882440,
       181385221])

In [ ]:
#as one could have expected, no empty IDs
skills['SOCIAL_PROFILE_ID'].isna().sum()

0

In [ ]:
#lets check how many single different skills we have
skills.groupby('SKILL').size().sort_values(ascending = False)

SKILL
SQL                               9656
Python                            6109
Machine Learning                  4433
Data Analysis                     4351
Data Science                      4294
                                  ... 
Materials Control                    1
Materials Design                     1
Materials Development                1
Materials Management Practices       1
trigonometry                         1
Length: 2769, dtype: int64

In [ ]:
#Trying with distinct, to see if we get 2769 as well
skills['SKILL'].nunique()

2769

In [ ]:
#check for empty skill records
skills['SKILL'].isna().sum()


1820

In [ ]:
#dropping empty records
skills.dropna(subset=['SKILL'], inplace=True)

In [ ]:
skills['SKILL'].isna().sum()

0

In [ ]:
skills.shape[0]

137409

In [ ]:
#check if the same skill has been reported twice for the same user --> no duplicates for the same user-skill couple
skills.groupby(['SOCIAL_PROFILE_ID','SKILL']).size().sort_values(ascending = False).loc[lambda x: x > 1]

Series([], dtype: int64)

In [ ]:
#all strings --> yes
skills[skills['SKILL'].map(type) == str].shape[0]

137409

In [ ]:
#explore records with missing gender
print(f'Nr of records with missing gender: {skills.loc[skills["PREDICTED_GENDER_ID"].isna(), "SOCIAL_PROFILE_ID"].shape[0]}')
#let's keep unique values
print(f'Nr of users with missing gender: {skills.loc[skills["PREDICTED_GENDER_ID"].isna(), "SOCIAL_PROFILE_ID"].drop_duplicates().shape[0]}')




Nr of records with missing gender: 3407
Nr of users with missing gender: 358


In [ ]:
#find average knowledge of each skill
user_size = skills['SOCIAL_PROFILE_ID'].nunique()
print(user_size)
skill_distr = skills.groupby(['SKILL']).size().apply(lambda x:x*100/user_size).sort_values(ascending = False)
print(skill_distr)

13664
SKILL
SQL                               70.667447
Python                            44.708724
Machine Learning                  32.442916
Data Analysis                     31.842799
Data Science                      31.425644
                                    ...    
Materials Control                  0.007319
Materials Design                   0.007319
Materials Development              0.007319
Materials Management Practices     0.007319
trigonometry                       0.007319
Length: 2769, dtype: float64


In [ ]:
#pareto approach: find the percentage the first quintile 20% of skills makes for --> see pareto analysis in the section below
skill_tally = skills['SKILL'].nunique()
print(skill_tally)
quintile = int(0.2*skill_tally)
skills_quintile = skills.groupby('SKILL').size().sort_values(ascending = False).iloc[:quintile].reset_index(name = 'COUNT')
print(skills_quintile)

2769
                    SKILL  COUNT
0                     SQL   9656
1                  Python   6109
2        Machine Learning   4433
3           Data Analysis   4351
4            Data Science   4294
..                    ...    ...
548                  Ruby     24
549          Apache Kafka     24
550  Voice over IP (VoIP)     24
551  Logistics Management     24
552            Child Care     24

[553 rows x 2 columns]


In [ ]:
#check how much the first 20% of skills accounts for --> 128142 records out of 139229
df_quintile = pd.DataFrame(skills_quintile)
sum_pareto = int(skills[skills['SKILL'].isin(df_quintile['SKILL'])].shape[0])
print(f'The first 1/5 of skills makes for {sum_pareto} skill records')
sum_skills = int(skills.shape[0])
print(f'Total nr of skill records (5/5) is {sum_skills}')
print(f'Percentage is {round(sum_pareto/sum_skills*100, 2)}')

The first 1/5 of skills makes for 126322 skill records
Total nr of skill records (5/5) is 137409
Percentage is 91.93


In [ ]:
#find how many skills are reported by less than 10 people --> 1842 out of 2769--> also useful for later
skills.groupby('SKILL').size().sort_values(ascending = False).loc[lambda x:x<10]

SKILL
Business Solutions                     9
MailChimp                              9
Service-Oriented Architecture (SOA)    9
Mediation                              9
Cell Culturing                         9
                                      ..
Materials Control                      1
Materials Design                       1
Materials Development                  1
Materials Management Practices         1
trigonometry                           1
Length: 1842, dtype: int64

# Text Analysis

In [ ]:

# Function to lemmatize & remove stopwords
def process_text(text):
    tokens = word_tokenize(text)
    stop_words_italian = set(stopwords.words('italian'))
    stop_words_english = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words_italian and word.lower() not in stop_words_english and word.isalpha()]

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return lemmatized_tokens

#create new df to map new skill labels to old skill labels
skill_sorted = skills.groupby(['SKILL']).size().sort_values(ascending = False).reset_index(name = 'COUNT')

# new column to store lemmatization results
skill_sorted['TOKEN_SKILL'] = skill_sorted['SKILL'].apply(process_text)


print(skill_sorted[['SKILL', 'TOKEN_SKILL']])


                               SKILL                       TOKEN_SKILL
0                                SQL                             [sql]
1                             Python                          [python]
2                   Machine Learning               [machine, learning]
3                      Data Analysis                  [data, analysis]
4                       Data Science                   [data, science]
...                              ...                               ...
2764               Materials Control               [material, control]
2765                Materials Design                [material, design]
2766           Materials Development           [material, development]
2767  Materials Management Practices  [material, management, practice]
2768                    trigonometry                    [trigonometry]

[2769 rows x 2 columns]


In [ ]:
#word_count
word_dict = {}
for i in skill_sorted['TOKEN_SKILL']:
  for j in i:
    if j in word_dict.keys():
      word_dict[j] = word_dict[j] + 1
    else:
      word_dict[j] = 1
df_word_dict2 = pd.DataFrame.from_dict(word_dict, orient='index', columns=['Value'])

In [ ]:
import pprint

In [ ]:
#first 30 words occurring most frequently
pprint.pprint(df_word_dict2.sort_values('Value',ascending=False).iloc[0:30])

               Value
management       125
analysis          76
design            65
system            60
development       48
data              47
sale              44
marketing         43
engineering       42
planning          34
knowledge         32
business          31
microsoft         28
service           26
industry          25
application       24
language          23
oracle            23
energy            23
process           22
testing           22
apache            22
information       20
control           20
software          19
financial         19
environmental     19
medical           19
programming       18
product           18


In [ ]:
#does not seem like lemmatization can really help in grouping equivalent skills together & reducing the total number

# EXPERIENCE - RAW TITLE JOIN

In [ ]:
#retrieving data title from the experience csv, in order to link each social profile to his/her data profession
#goal is to perform the skill analysis "tailored" for each data profession
current_directory = os.getcwd()
EDU_TOK = pd.read_csv('EDUCATION_TOK.csv', sep = ';')

In [ ]:
EDU_TOK.head()

SOCIAL_PROFILE_ID      DATA_TITLE
0          588127715  Data scientist
1          318907477  Data scientist
2          558428076  Data scientist
3          229581338    Data analyst
4           88465187  Data scientist

In [ ]:
EDU_TOK = EDU_TOK.rename(columns = {'SOCIAL_PROFILE_ID' : 'SOCIAL_TOK'})

In [ ]:
EDU_TOK.columns

Index(['SOCIAL_TOK', 'DATA_TITLE'], dtype='object')

In [ ]:
#great! if group by ID,DATA_TITLE & ID only deliver the same total, it must mean that we do not have different job titles for the same user record!
print(EDU_TOK.groupby(['SOCIAL_TOK','DATA_TITLE']).count().shape[0])
print(EDU_TOK.groupby(['SOCIAL_TOK']).count().shape[0])

15484
15484


In [ ]:
#could have used unique()as well
EDU_TOK_GROUPED = EDU_TOK.groupby(['SOCIAL_TOK']).first()

In [ ]:
skills_w_id = pd.merge(skills, EDU_TOK_GROUPED, left_on='SOCIAL_PROFILE_ID', right_on='SOCIAL_TOK', how='left')

In [ ]:
#ok, all records are assigned an ID now
skills_w_id[skills_w_id['DATA_TITLE'].isna()==True].shape[0]

0

In [ ]:
#saving and exporting df
skills_w_id.to_csv('/content/skills_w_id.csv', index=True)
files.download('/content/skills_w_id.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# File CSV in un DataFrame pandas
uploaded = files.upload()
skills_w_id= pd.read_csv('skills_w_id.csv', sep = ',')

Saving skills_w_id.csv to skills_w_id.csv


#SKILLS NEW IMPORT

In [ ]:
skills_w_id.tail()

Unnamed: 0  SOCIAL_PROFILE_ID  PREDICTED_GENDER_ID  \
139224      139224          408849516                  1.0   
139225      139225          340141268                  1.0   
139226      139226          336184901                  1.0   
139227      139227          336789478                  1.0   
139228      139228          336690280                  1.0   

                                                RAW_TITLE        SKILL  \
139224                 Geospatial Data Scientist at at800  Data Mining   
139225  Software Data Analyst presso PHPower - Gruppo ...   JavaScript   
139226     Senior Data Scientist @ Data Reply IT | PoliMI            R   
139227                 Data Analyst presso WISHINNOVATION          SQL   
139228                                     Data Scientist          SQL   

       IS_PARSED_FROM_PROFILE_SUMMARY IS_INFERRED_FROM_EXPERIENCE  \
139224                         FALSCH                      FALSCH   
139225                         FALSCH                        WAHR   
139226                         FALSCH                      FALSCH   
139227                         FALSCH                        WAHR   
139228                         FALSCH                        WAHR   

       IS_PARSED_FROM_SKILLS_SECTION      DATA_TITLE  
139224                          WAHR  Data scientist  
139225                          WAHR    Data analyst  
139226                          WAHR  Data scientist  
139227                        FALSCH    Data analyst  
139228                        FALSCH  Data scientist

In [ ]:
skills_w_id.shape[0]

139229

In [ ]:
dict_jobsub = dict(skills_w_id.drop_duplicates('SOCIAL_PROFILE_ID').groupby('DATA_TITLE').size())

In [ ]:
print(dict_jobsub)

{'Altro': 145, 'Data analyst': 6511, 'Data architect': 76, 'Data engineer': 2668, 'Data scientist': 6084}


In [ ]:
skills_by_job = skills_w_id.groupby(['DATA_TITLE','SKILL']).size().sort_values(ascending = False)

In [ ]:
#saving and exporting df
skills_by_job.to_csv('/content/skills_by_job.csv', index=True)
files.download('/content/skills_by_job.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **LEVENSHTEIN**

In [ ]:
#trying levenstein function to group together skills that are lexically very close e.g. 'Account Management', 'Accounting Management'

In [ ]:
pip install fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.9 MB/s eta 0:00:00


In [ ]:
#apply levenshtein distance to group together lexically similar words
#starting from an empty dictionary, create a first rep_skill:group element. Evaluate for each skill in the argument if the similarity between skill, rep_skill is above the set threshold. If so, add the skill to the group. If no group fit, create a new group. Start over.
from fuzzywuzzy import fuzz

def skill_lev(skills: list, sim_threshold: float):
    grouped_skills = {}
    for skill in skills:
        if not grouped_skills:
            # If the dictionary is empty, create the first group with the first skill
            grouped_skills[skill] = [skill]
            continue

        group_found = False
        for representative_skill, skill_group in grouped_skills.items():
            similarity_score = fuzz.ratio(skill, representative_skill) / 100.0
            if similarity_score >= sim_threshold and similarity_score != 1:
                skill_group.append(skill)
                group_found = True
                print(representative_skill, skill_group)
                break
        if not group_found:
            representative_skill = skill  # Set the representative skill as the current skill
            grouped_skills[skill] = [skill]
    return grouped_skills

In [ ]:
#let's order the skills by occurrence
#I want more frequently reported skills to be grouped less frequently reported skills to, and not the other way around
# Call the function to group similar skills
similarity_threshold = 0.9
skill_sorted = skills.groupby(['SKILL']).size().sort_values(ascending = False).reset_index(name = 'COUNT')
grouped_skills_dict = skill_lev(skill_sorted['SKILL'], similarity_threshold)


Team Management ['Team Management', 'Tag Management']
Business Development ['Business Development', 'New Business Development']
Geographic Information System (GIS) ['Geographic Information System (GIS)', 'Geographic Information System (GIS) Data']
Quantitative Research ['Quantitative Research', 'Qualitative Research']
Content Management ['Content Management', 'Web Content Management']
Product Management ['Product Management', 'Production Management']
Project Management ['Project Management', 'Web Project Management']
ISO 9001 Standards ['ISO 9001 Standards', 'ISO 14001 Standards']
Store Management ['Store Management', 'Stress Management']
Computer Aided Manufacturing (CAM) ['Computer Aided Manufacturing (CAM)', 'Computer-Aided Manufacturing (CAM)']
Retail Sales ['Retail Sales', 'Rental Sales']
Microsoft Dynamics ['Microsoft Dynamics', 'Microsoft Dynamics NAV']
Microsoft Dynamics ['Microsoft Dynamics', 'Microsoft Dynamics NAV', 'Microsoft Dynamics AX']
Healthcare Information ['Healthcar

In [ ]:
print(len(grouped_skills_dict))
print(grouped_skills_dict)

2742
{'SQL': ['SQL'], 'Python': ['Python'], 'Machine Learning': ['Machine Learning'], 'Data Analysis': ['Data Analysis'], 'Data Science': ['Data Science'], 'Project Management': ['Project Management', 'Web Project Management', 'HR Project Management'], 'Java': ['Java'], 'Big Data': ['Big Data'], 'Microsoft Office': ['Microsoft Office'], 'Extraction Transformation and Loading (ETL)': ['Extraction Transformation and Loading (ETL)'], 'Data Engineering': ['Data Engineering'], 'AWS Redshift': ['AWS Redshift'], 'Pipeline (Computing)': ['Pipeline (Computing)'], 'Software Development': ['Software Development'], 'Budgeting': ['Budgeting'], 'Microsoft Excel': ['Microsoft Excel'], 'Customer Service': ['Customer Service'], 'Software Engineering': ['Software Engineering'], 'Business Analysis': ['Business Analysis'], 'Sales': ['Sales'], 'Business Intelligence': ['Business Intelligence'], 'Marketing': ['Marketing', 'E-Marketing'], 'Teaching': ['Teaching'], 'JavaScript': ['JavaScript'], 'Research': ['

In [ ]:
reverse_skills_dict = {skill: token for token, skills in grouped_skills_dict.items() for skill in skills}

# Apply the mapping to create a new column 'TOKEN_SKILL' in the DataFrame
skill_sorted['TOKEN_SKILL'] = skill_sorted['SKILL'].apply(lambda skill: reverse_skills_dict.get(skill))

In [ ]:
print(skill_sorted[['SKILL','TOKEN_SKILL']].loc[skill_sorted['SKILL']!=skill_sorted['TOKEN_SKILL']])

                                         SKILL  \
246                             Tag Management   
266                   New Business Development   
355   Geographic Information System (GIS) Data   
377                       Qualitative Research   
559                     Web Content Management   
672                      Production Management   
678                     Web Project Management   
972                        ISO 14001 Standards   
1234                         Stress Management   
1260        Computer-Aided Manufacturing (CAM)   
1268                              Rental Sales   
1334                    Microsoft Dynamics NAV   
1533                     Microsoft Dynamics AX   
1583                        Health Information   
1646     Type 1 Diabetes Diagnosis / Treatment   
1776                        Channel Management   
1925                                 Basel III   
1955                   New Product Development   
2174                         Sample Management   


In [ ]:
#exporting dict as a csv so I can explore it offline
skill_sorted.to_csv('/content/skill_sorted.csv', index=True)
files.download('/content/skill_sorted.csv')

NameError: ignored

In [ ]:
#after reviewing the first clustering of skills performed by Levenstein function, let's reupload the skills mapping and try other forms of grouping
uploaded = files.upload()
skills_sorted_2 = pd.read_csv('./skill_ord.csv', sep = ';')

Saving skill_ord.numbers to skill_ord.numbers


# SBERT

In [ ]:
#levenstein function is good to combine slightly different strings together. by deriving semantically meaningful sentence embeddings that can be compared using cosine-similarity.

In [ ]:
uploaded = files.upload()
skills_sorted_2 = pd.read_csv('./skill_ord.csv', sep = ';')

Saving skill_ord.csv to skill_ord.csv


In [ ]:
pip install transformers

In [ ]:
pip install sentence-transformers

In [ ]:
pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dask.distributed import Client

def sbert_embedding(model, text):
    return model.encode(text, convert_to_tensor=True)

def skill_sbert(skills: list, sim_threshold: float):
    model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
    grouped_skills = {}
    client = Client()

    # Precompute embeddings for unique skill texts
    unique_skills = list(set(skills))
    embeddings_dict = {skill: sbert_embedding(model, skill) for skill in unique_skills}

    for skill in tqdm(skills, desc="Processing skills", leave=False):
        embedding1 = embeddings_dict[skill]

        group_found = False
        for representative_skill, skill_group in grouped_skills.items():
            embedding2 = embeddings_dict[representative_skill]
            similarity_score = cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0))[0][0]

            if similarity_score >= sim_threshold and similarity_score != 1:
                skill_group.append(skill)
                group_found = True
                break

        if not group_found:
            grouped_skills[skill] = [skill]

    client.close()
    return grouped_skills

In [ ]:
#doing some trials with Sbert model, to find a reasonable similarity threshold
from scipy.spatial.distance import cosine
string1 = 'Help'
string2 = 'Support'
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
embed1 = sbert_embedding(model, string1)
embed2 =  sbert_embedding(model, string2)
similarity_score = 1 - cosine(embed1, embed2)
print(f"Semantic similarity between '{string1}' and '{string2}': {similarity_score}")

Semantic similarity between 'Help' and 'Support': 0.6967057585716248


In [ ]:
similarity_threshold = 0.9
sbert_skills_dict = skill_sbert(skills_sorted_2['TOKEN_SKILL'], similarity_threshold)

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:37775
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35739'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46247'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33583'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37369'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45901', name: 2, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45901
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48170
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:41057', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41057
INFO:distributed.core:Starting est

KeyboardInterrupt: ignored

INFO:distributed.core:Connection to tcp://127.0.0.1:48180 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:41057', name: 0, status: running, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1692821832.3186305')
INFO:distributed.core:Connection to tcp://127.0.0.1:48188 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:34647', name: 3, status: running, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1692821832.32528')
INFO:distributed.scheduler:Lost all workers


In [ ]:
reverse_skills_dict = {skill: token for token, skills in sbert_skills_dict.items() for skill in skills}
skills_sorted_2['SBERT_SKILL'] = skills_sorted_2['TOKEN_SKILL'].apply(lambda skill: reverse_skills_dict.get(skill))
skills_sorted_2.to_csv('/content/skill_sbert.csv', index=True)
files.download('/content/skill_sbert.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(len(sbert_skills_dict))
print(sbert_skills_dict)

# Skills Reduction

In [ ]:
skill_sbert = skills_sorted_2.copy()

In [ ]:
print(skill_sbert)

      Unnamed: 0.1  Unnamed: 0                 SKILL  COUNT  \
0             2391           0                   SQL   9656   
1             2073           1                Python   6109   
2             1479           2      Machine Learning   4433   
3              613           3         Data Analysis   4351   
4              635           4          Data Science   4294   
...            ...         ...                   ...    ...   
2764          2684        2085  Warehouse Operations      1   
2765          2726        2132      Wildlife Biology      1   
2766          2760        2111                  YAML      1   
2767          2766        2160                Zenoss      1   
2768          2767        2163                Zigbee      1   

               TOKEN_SKILL       SBERT_SKILL  
0                      SQL               SQL  
1                   Python            Python  
2         Machine Learning  Machine Learning  
3            Data Analysis     Data Analysis  
4       

In [ ]:
#after reviewing the first clustering of skills performed by Sbert function, let's reupload the skills mapping and try other forms of grouping
uploaded = files.upload()
skills_by_job= pd.read_csv('./skills_by_job.csv', sep = ',')

KeyboardInterrupt: ignored

In [ ]:
print(skills_by_job)

In [ ]:
skills_by_job_sbert = pd.merge(skills_by_job, skill_sbert, left_on='SKILL', right_on='SKILL', how='left')

In [ ]:
print(skills_by_job_sbert)

In [ ]:
#file was evaluated offline and some values were filled/updated manually
uploaded = files.upload()
skills_map_final= pd.read_csv('./skills_map_final.csv', sep = ';')

Saving skills_map_final.csv to skills_map_final.csv


In [ ]:

skill_map = skills_map_final.groupby(['SKILL_x', 'SBERT_SKILL']).first().reset_index()[['SKILL_x', 'SBERT_SKILL']]

In [ ]:
print(skill_map.drop_duplicates('SBERT_SKILL'))

                                        SKILL_x  \
0                                          .NET   
1                                21 CFR Part 11   
2                                  3D Animation   
3     3D Printing / Additive Manufacturing (AM)   
4                                 3D Studio Max   
...                                         ...   
2764                                   microRNA   
2765                                 phpMyAdmin   
2766                                       qPCR   
2767                                    tcpdump   
2768                               trigonometry   

                                    SBERT_SKILL  
0                                          .NET  
1                                21 CFR Part 11  
2                                  3D Animation  
3     3D Printing / Additive Manufacturing (AM)  
4                                 3D Studio Max  
...                                         ...  
2764                                 

In [ ]:
#so we went down from 2769 skills to 2471, good!

In [ ]:
#downloading final mapping
skill_map.to_csv('/content/skill_map.csv', index=True)
files.download('/content/skill_map.csv')

NameError: ignored

# Skill distribution - by data profession

In [ ]:
#next step is to reduce the nr of skills not by similarity, but based on their frequency. Since the size of the single data profession samples differs a lot (TOT(Data Analysts) >>> TOT(Data Architect))
#,the frequency of the skills should be studied separately for the single professions. I.E. if the Skill "Data Architecture" has only 30 records but all for the Data Architect Subgroups (<100),
#the skill should be considered relveant to describe the skillset of Data Architect (even if "under-represented" in the total sample)

In [ ]:
import pandas as pd
#after reviewing the first clustering of skills performed by Levenstein function, let's reupload the skills mapping and try other forms of grouping
uploaded = files.upload()
skills_by_job_sbert= pd.read_csv('./skills_by_job_sbert_1.csv', sep = ',')

Saving skills_by_job_sbert_1.csv to skills_by_job_sbert_1 (1).csv


In [ ]:
result = skills_by_job_sbert[['DATA_TITLE', 'SBERT_SKILL', '0']].groupby(['DATA_TITLE', 'SBERT_SKILL']).sum().sort_values(by='0', ascending=False)
result_skill = result.reset_index()
result_skill.columns = ['DATA_TITLE', 'SBERT_SKILL', 'SUM_0']
print(result_skill)


          DATA_TITLE            SBERT_SKILL  SUM_0
0     Data scientist                    SQL   4379
1     Data scientist                 Python   3694
2     Data scientist       Machine Learning   3612
3     Data scientist           Data Science   3574
4       Data analyst                    SQL   3453
...              ...                    ...    ...
5853   Data engineer                 Arabic      1
5854   Data engineer             ArcObjects      1
5855   Data engineer  Architectural History      1
5856   Data engineer     Archival Materials      1
5857   Data engineer    Category Management      1

[5858 rows x 3 columns]


In [ ]:
#update result_skill['SBERT_SKILL'] as per latest mapping (where some values where modified manually)
result_skill = pd.merge(result_skill, skill_map, left_on='SBERT_SKILL', right_on='SKILL_x', how='inner')

In [ ]:
#check where after the last update the skills differ
result_skill[result_skill['SKILL_x']!=result_skill['SBERT_SKILL_y']]

DATA_TITLE              SBERT_SKILL_x  SUM_0  \
714     Data analyst              Team Building     69   
715   Data scientist              Team Building     29   
716    Data engineer              Team Building     15   
717            Altro              Team Building      1   
772   Data scientist       Statistical Analysis     63   
...              ...                        ...    ...   
5471    Data analyst   Treatment Recommendation      1   
5478    Data analyst      Strategic Development      1   
5479   Data engineer      Strategic Development      1   
5480    Data analyst  Strategic Recommendations      1   
5731   Data engineer          Welding Equipment      1   

                        SKILL_x             SBERT_SKILL_y  
714               Team Building  Teamwork / Collaboration  
715               Team Building  Teamwork / Collaboration  
716               Team Building  Teamwork / Collaboration  
717               Team Building  Teamwork / Collaboration  
772        Statistical Analysis                Statistics  
...                         ...                       ...  
5471   Treatment Recommendation        Treatment Planning  
5478      Strategic Development        Strategic Planning  
5479      Strategic Development        Strategic Planning  
5480  Strategic Recommendations        Strategic Planning  
5731          Welding Equipment                   Welding  

[85 rows x 5 columns]

In [ ]:
#see above
print(dict_jobsub)

{'Altro': 145, 'Data analyst': 6511, 'Data architect': 76, 'Data engineer': 2668, 'Data scientist': 6084}


In [ ]:
#skill %& knowledge by job/title
result_skill['%by_job'] = (result_skill['SUM_0'] / result_skill['DATA_TITLE'].map(dict_jobsub)).apply(lambda x: round(x * 100, 2))

In [ ]:
print(result_skill)

          DATA_TITLE    SBERT_SKILL_x  SUM_0            SKILL  \
0     Data scientist              SQL   4379              SQL   
1       Data analyst              SQL   3453              SQL   
2      Data engineer              SQL   2304              SQL   
3              Altro              SQL     78              SQL   
4     Data architect              SQL     75              SQL   
...              ...              ...    ...              ...   
5853   Data engineer  Apache ActiveMQ      1  Apache ActiveMQ   
5854   Data engineer      Apache Flex      1      Apache Flex   
5855   Data engineer       Apache Ivy      1       Apache Ivy   
5856   Data engineer     Appcelerator      1     Appcelerator   
5857   Data engineer    Apple Keynote      1    Apple Keynote   

        SBERT_SKILL_y  %by_job  
0                 SQL    71.98  
1                 SQL    53.03  
2                 SQL    86.36  
3                 SQL    53.79  
4                 SQL    98.68  
...               ...

In [ ]:
#result_skill = result_skill.drop(columns = 'Unnamed: 0' )

KeyError: ignored

In [ ]:
result_skill = result_skill.rename(columns = {'SKILL_x':'SKILL'})
result_skill = result_skill.rename(columns = {'SBERT_SKILL_y':'SBERT_SKILL'})

In [ ]:
print(result_skill)

          DATA_TITLE    SBERT_SKILL_x  SUM_0            SKILL  \
0     Data scientist              SQL   4379              SQL   
1       Data analyst              SQL   3453              SQL   
2      Data engineer              SQL   2304              SQL   
3              Altro              SQL     78              SQL   
4     Data architect              SQL     75              SQL   
...              ...              ...    ...              ...   
5853   Data engineer  Apache ActiveMQ      1  Apache ActiveMQ   
5854   Data engineer      Apache Flex      1      Apache Flex   
5855   Data engineer       Apache Ivy      1       Apache Ivy   
5856   Data engineer     Appcelerator      1     Appcelerator   
5857   Data engineer    Apple Keynote      1    Apple Keynote   

          SBERT_SKILL  %by_job  
0                 SQL    71.98  
1                 SQL    53.03  
2                 SQL    86.36  
3                 SQL    53.79  
4                 SQL    98.68  
...               ...

In [ ]:
#now the representativity of each skill is weighted on the sample size for each profession
#let's see what we are left with if we delete <1% records --> good! as we hinted the skil distribution is very "pareto-like" --> in the section below further evaluations to back up the statement
result_skill[result_skill['%by_job']>1].drop_duplicates('SBERT_SKILL')

DATA_TITLE  SUM_0                             SKILL  \
0     Data scientist   4379                               SQL   
5     Data scientist   3694                            Python   
10    Data scientist   3612                  Machine Learning   
15    Data scientist   3574                      Data Science   
20      Data analyst   2663                     Data Analysis   
...              ...    ...                               ...   
3376  Data architect      2     Test Driven Development (TDD)   
3436  Data architect      2              Lifecycle Management   
3682  Data architect      2   Network Management Center (NMC)   
3804  Data architect      2  Application Lifecycle Management   
4453  Data architect      2       Continuous Integration (CI)   

                           SBERT_SKILL  %by_job  
0                                  SQL     72.0  
5                               Python     61.0  
10                    Machine Learning     59.0  
15                        Data Science     59.0  
20                       Data Analysis     41.0  
...                                ...      ...  
3376     Test Driven Development (TDD)      3.0  
3436              Lifecycle Management      3.0  
3682   Network Management Center (NMC)      3.0  
3804  Application Lifecycle Management      3.0  
4453       Continuous Integration (CI)      3.0  

[243 rows x 5 columns]

# **DATA** **SCIENTIST**

In [ ]:
#applying pareto analysis for the "data scientist subset"

In [ ]:
job_title_sums = result_skill.groupby('DATA_TITLE')['SUM_0'].sum()
data_scientist_sum = job_title_sums.get('Data scientist', 0)  # Get the sum for 'Data scientist', default to 0 if not found
print(data_scientist_sum)

52425


In [ ]:
data_scientist = result_skill[result_skill['DATA_TITLE'] == 'Data scientist']
data_shape = data_scientist.shape[0]
print(data_shape)

1741


In [ ]:
#pareto approach: find the percentage the first quintile 20% of skills makes for
data_scientist_sum = job_title_sums.get('Data scientist', 0)
quintile = int(0.2*data_shape)
print(quintile)
skills_quintile_d_sc  = data_scientist.iloc[:quintile]
print(skills_quintile_d_sc)

348
          DATA_TITLE      SBERT_SKILL_x  SUM_0              SKILL  \
0     Data scientist                SQL   4379                SQL   
5     Data scientist             Python   3694             Python   
10    Data scientist   Machine Learning   3612   Machine Learning   
15    Data scientist       Data Science   3574       Data Science   
21    Data scientist      Data Analysis   1313      Data Analysis   
...              ...                ...    ...                ...   
1449  Data scientist         Recruiting      6         Recruiting   
1453  Data scientist           Cloudera      9           Cloudera   
1456  Data scientist  Advertising Sales     10  Advertising Sales   
1458  Data scientist      Biotechnology     24      Biotechnology   
1462  Data scientist     Legal Research     24     Legal Research   

            SBERT_SKILL  %by_job  
0                   SQL    71.98  
5                Python    60.72  
10     Machine Learning    59.37  
15         Data Science    

In [ ]:
#DATA SCIENTIST
#check how much the first 20% of skills accounts for --> 47041 records out of 52425
df_quintile = pd.DataFrame(skills_quintile_d_sc)
sum_pareto = int(skills_quintile_d_sc['SUM_0'].sum())
print(f'The first 1/5 of skills makes for {sum_pareto} skill records')
print(f'Total nr of skill records (5/5) is {data_scientist_sum}')
print(f'Percentage is {round(sum_pareto/data_scientist_sum*100, 2)}')

The first 1/5 of skills makes for 47041 skill records
Total nr of skill records (5/5) is 52425
Percentage is 89.73


In [ ]:
data_scientist['PARETO'] = data_scientist['SBERT_SKILL'].isin(skills_quintile_d_sc['SBERT_SKILL'])

<ipython-input-90-fd2e10bd992d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_scientist['PARETO'] = data_scientist['SBERT_SKILL'].isin(skills_quintile_d_sc['SBERT_SKILL'])


In [ ]:
print(data_scientist)

          DATA_TITLE           SBERT_SKILL_x  SUM_0                   SKILL  \
0     Data scientist                     SQL   4379                     SQL   
5     Data scientist                  Python   3694                  Python   
10    Data scientist        Machine Learning   3612        Machine Learning   
15    Data scientist            Data Science   3574            Data Science   
21    Data scientist           Data Analysis   1313           Data Analysis   
...              ...                     ...    ...                     ...   
5756  Data scientist              Audiometry      1              Audiometry   
5757  Data scientist  Augmented Reality (AR)      1  Augmented Reality (AR)   
5758  Data scientist          Authentication      1          Authentication   
5760  Data scientist            Autoencoders      1            Autoencoders   
5761  Data scientist        Automation Tools      1        Automation Tools   

                 SBERT_SKILL  %by_job  PARETO  
0  

# **DATA** **ARCHITECT**

In [ ]:
#applying pareto analysis for the "data architect subset"

In [ ]:
job_title_sums = result_skill.groupby('DATA_TITLE')['SUM_0'].sum()
data_architect_sum = job_title_sums.get('Data architect', 0)  # Get the sum for 'Data architect', default to 0 if not found
print(data_architect_sum)

1356


In [ ]:
data_architect = result_skill[result_skill['DATA_TITLE'] == 'Data architect']
data_shape = data_architect.shape[0]
print(data_shape)

361


In [ ]:
#pareto approach: find the percentage the first quintile 20% of skills makes for
quintile = int(0.2*data_shape)
print(quintile)
skills_quintile_d_ar = data_architect.iloc[:quintile]
print(skills_quintile_d_ar)

72
         DATA_TITLE     SBERT_SKILL_x  SUM_0             SKILL  \
4    Data architect               SQL     75               SQL   
9    Data architect            Python     30            Python   
14   Data architect  Machine Learning     16  Machine Learning   
19   Data architect      Data Science      9      Data Science   
24   Data architect     Data Analysis      7     Data Analysis   
..              ...               ...    ...               ...   
424  Data architect          Planning      8          Planning   
430  Data architect   Risk Management      1   Risk Management   
434  Data architect   Data Management      6   Data Management   
440  Data architect     Oracle PL/SQL      5     Oracle PL/SQL   
448  Data architect     Apache Hadoop      4     Apache Hadoop   

          SBERT_SKILL  %by_job  
4                 SQL    98.68  
9              Python    39.47  
14   Machine Learning    21.05  
19       Data Science    11.84  
24      Data Analysis     9.21  
..    

In [ ]:
data_architect['PARETO'] = data_architect['SBERT_SKILL'].isin(skills_quintile_d_ar['SBERT_SKILL'])

<ipython-input-95-7fdf5319398c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_architect['PARETO'] = data_architect['SBERT_SKILL'].isin(skills_quintile_d_ar['SBERT_SKILL'])


In [ ]:
#DATA ARCHITECT
#check how much the first 20% of skills accounts for
df_quintile = pd.DataFrame(skills_quintile_d_ar )
sum_pareto = int(df_quintile['SUM_0'].sum())
print(f'The first 1/5 of skills makes for {sum_pareto} skill records')
print(f'Total nr of skill records (5/5) is {data_architect_sum}')
print(f'Percentage is {round(sum_pareto/data_architect_sum*100, 2)}')

The first 1/5 of skills makes for 765 skill records
Total nr of skill records (5/5) is 1356
Percentage is 56.42


In [ ]:
print(data_architect)

          DATA_TITLE                       SBERT_SKILL_x  SUM_0  \
4     Data architect                                 SQL     75   
9     Data architect                              Python     30   
14    Data architect                    Machine Learning     16   
19    Data architect                        Data Science      9   
24    Data architect                       Data Analysis      7   
...              ...                                 ...    ...   
5799  Data architect                              Quartz      1   
5803  Data architect                          Windows CE      1   
5819  Data architect          Electronic Medical Records      1   
5820  Data architect  Application Performance Management      1   
5821  Data architect                    Atlassian Bamboo      1   

                                   SKILL                         SBERT_SKILL  \
4                                    SQL                                 SQL   
9                                 P

# **DATA** **ENGINEER**

In [ ]:
#applying pareto analysis for the "data engineer subset"

In [ ]:
job_title_sums = result_skill.groupby('DATA_TITLE')['SUM_0'].sum()
data_engineer_sum = job_title_sums.get('Data engineer', 0)  # Get the sum for 'Data architect', default to 0 if not found
print(data_engineer_sum)

32709


In [ ]:
data_engineer = result_skill[result_skill['DATA_TITLE'] == 'Data engineer']
data_shape = data_engineer.shape[0]
print(data_shape)

1389


In [ ]:
#pareto approach: find the percentage the first quintile 20% of skills makes for
quintile = int(0.2*data_shape)
print(quintile)
skills_quintile_d_en = data_engineer.iloc[:quintile]
print(skills_quintile_d_en)

277
         DATA_TITLE                      SBERT_SKILL_x  SUM_0  \
2     Data engineer                                SQL   2304   
6     Data engineer                             Python   1749   
11    Data engineer                   Machine Learning    400   
17    Data engineer                       Data Science    315   
22    Data engineer                      Data Analysis    337   
...             ...                                ...    ...   
1189  Data engineer                 Forklift Operation      4   
1194  Data engineer                        Forecasting      5   
1196  Data engineer  JavaScript Object Notation (JSON)     36   
1202  Data engineer                Operations Analysis      4   
1205  Data engineer                  Wealth Management      2   

                                  SKILL                        SBERT_SKILL  \
2                                   SQL                                SQL   
6                                Python                    

In [ ]:
data_engineer['PARETO'] = data_engineer['SBERT_SKILL'].isin(skills_quintile_d_en['SBERT_SKILL'])

<ipython-input-101-217d21834fc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_engineer['PARETO'] = data_engineer['SBERT_SKILL'].isin(skills_quintile_d_en['SBERT_SKILL'])


In [ ]:
#DATA ENGINEER
#check how much the first 20% of skills accounts for
df_quintile = pd.DataFrame(skills_quintile_d_en)
sum_pareto = int(df_quintile['SUM_0'].sum())
print(f'The first 1/5 of skills makes for {sum_pareto} skill records')
print(f'Total nr of skill records (5/5) is {data_engineer_sum}')
print(f'Percentage is {round(sum_pareto/data_engineer_sum*100, 2)}')

The first 1/5 of skills makes for 28416 skill records
Total nr of skill records (5/5) is 32709
Percentage is 86.88


In [ ]:
print(data_engineer)

         DATA_TITLE     SBERT_SKILL_x  SUM_0             SKILL  \
2     Data engineer               SQL   2304               SQL   
6     Data engineer            Python   1749            Python   
11    Data engineer  Machine Learning    400  Machine Learning   
17    Data engineer      Data Science    315      Data Science   
22    Data engineer     Data Analysis    337     Data Analysis   
...             ...               ...    ...               ...   
5853  Data engineer   Apache ActiveMQ      1   Apache ActiveMQ   
5854  Data engineer       Apache Flex      1       Apache Flex   
5855  Data engineer        Apache Ivy      1        Apache Ivy   
5856  Data engineer      Appcelerator      1      Appcelerator   
5857  Data engineer     Apple Keynote      1     Apple Keynote   

           SBERT_SKILL  %by_job  PARETO  
2                  SQL    86.36    True  
6               Python    65.55    True  
11    Machine Learning    14.99    True  
17        Data Science    11.81    True

# **DATA** **ANALYST**

In [ ]:
#applying pareto analysis for the "data analyst subset"

In [ ]:
job_title_sums = result_skill.groupby('DATA_TITLE')['SUM_0'].sum()
data_analyst_sum = job_title_sums.get('Data analyst', 0)  # Get the sum for 'Data architect', default to 0 if not found
print(data_analyst_sum)

48837


In [ ]:
data_analyst = result_skill[result_skill['DATA_TITLE'] == 'Data analyst']
data_shape = data_analyst.shape[0]
print(data_shape)

1935


In [ ]:
#pareto approach: find the percentage the first quintile 20% of skills makes for
quintile = int(0.2*data_shape)
print(quintile)
skills_quintile_d_an= data_analyst.iloc[:quintile]
print(skills_quintile_d_an)

387
        DATA_TITLE        SBERT_SKILL_x  SUM_0                SKILL  \
1     Data analyst                  SQL   3453                  SQL   
7     Data analyst               Python    598               Python   
12    Data analyst     Machine Learning    384     Machine Learning   
16    Data analyst         Data Science    375         Data Science   
20    Data analyst        Data Analysis   2663        Data Analysis   
...            ...                  ...    ...                  ...   
1590  Data analyst  Digital Advertising     21  Digital Advertising   
1593  Data analyst           Child Care     21           Child Care   
1598  Data analyst           LexisNexis      4           LexisNexis   
1600  Data analyst        Esri Software     21        Esri Software   
1606  Data analyst       Zend Framework     10       Zend Framework   

              SBERT_SKILL  %by_job  
1                     SQL    53.03  
7                  Python     9.18  
12       Machine Learning     5.

In [ ]:
data_analyst['PARETO'] = data_analyst['SBERT_SKILL'].isin(skills_quintile_d_an['SBERT_SKILL'])

<ipython-input-108-ad344751c570>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_analyst['PARETO'] = data_analyst['SBERT_SKILL'].isin(skills_quintile_d_an['SBERT_SKILL'])


In [ ]:
#DATA ANALYST
#check how much the first 20% of skills accounts for
df_quintile = pd.DataFrame(skills_quintile_d_an)
sum_pareto = int(df_quintile['SUM_0'].sum())
print(f'The first 1/5 of skills makes for {sum_pareto} skill records')
print(f'Total nr of skill records (5/5) is {data_analyst_sum}')
print(f'Percentage is {round(sum_pareto/data_analyst_sum*100, 2)}')

The first 1/5 of skills makes for 42702 skill records
Total nr of skill records (5/5) is 48837
Percentage is 87.44


In [ ]:
print( data_analyst)

        DATA_TITLE            SBERT_SKILL_x  SUM_0                    SKILL  \
1     Data analyst                      SQL   3453                      SQL   
7     Data analyst                   Python    598                   Python   
12    Data analyst         Machine Learning    384         Machine Learning   
16    Data analyst             Data Science    375             Data Science   
20    Data analyst            Data Analysis   2663            Data Analysis   
...            ...                      ...    ...                      ...   
5814  Data analyst                 Virology      1                 Virology   
5815  Data analyst  Vital Signs Measurement      1  Vital Signs Measurement   
5816  Data analyst          Waste reduction      1          Waste reduction   
5817  Data analyst           Weatherization      1           Weatherization   
5818  Data analyst            Web Standards      1            Web Standards   

                  SBERT_SKILL  %by_job  PARETO  
1 

# PARETO - CONCATENATION

In [ ]:
#now we can drop not representative skills but considering also the representativity within each data profession!

In [ ]:
#CONCATENATION
result_pareto = pd.concat([data_analyst,data_scientist, data_engineer, data_architect], axis = 0)

In [ ]:
print(result_pareto)

          DATA_TITLE                       SBERT_SKILL_x  SUM_0  \
1       Data analyst                                 SQL   3453   
7       Data analyst                              Python    598   
12      Data analyst                    Machine Learning    384   
16      Data analyst                        Data Science    375   
20      Data analyst                       Data Analysis   2663   
...              ...                                 ...    ...   
5799  Data architect                              Quartz      1   
5803  Data architect                          Windows CE      1   
5819  Data architect          Electronic Medical Records      1   
5820  Data architect  Application Performance Management      1   
5821  Data architect                    Atlassian Bamboo      1   

                                   SKILL                         SBERT_SKILL  \
1                                    SQL                                 SQL   
7                                 P

In [ ]:
#trying different combinatioon of metrics to check the the resulting subset of skills

In [ ]:
filtered_result = result_pareto.loc[(result_pareto['PARETO'] == True) | (result_pareto['%by_job'] > 1)].drop_duplicates('SBERT_SKILL')
print(filtered_result)

          DATA_TITLE                       SBERT_SKILL_x  SUM_0  \
1       Data analyst                                 SQL   3453   
7       Data analyst                              Python    598   
12      Data analyst                    Machine Learning    384   
16      Data analyst                        Data Science    375   
20      Data analyst                       Data Analysis   2663   
...              ...                                 ...    ...   
5799  Data architect                              Quartz      1   
5803  Data architect                          Windows CE      1   
5819  Data architect          Electronic Medical Records      1   
5820  Data architect  Application Performance Management      1   
5821  Data architect                    Atlassian Bamboo      1   

                                   SKILL                         SBERT_SKILL  \
1                                    SQL                                 SQL   
7                                 P

In [ ]:
filtered_result = result_pareto.loc[result_pareto['PARETO'] == True].drop_duplicates('SBERT_SKILL')
print(filtered_result)

        DATA_TITLE        SBERT_SKILL_x  SUM_0                SKILL  \
1     Data analyst                  SQL   3453                  SQL   
7     Data analyst               Python    598               Python   
12    Data analyst     Machine Learning    384     Machine Learning   
16    Data analyst         Data Science    375         Data Science   
20    Data analyst        Data Analysis   2663        Data Analysis   
...            ...                  ...    ...                  ...   
1590  Data analyst  Digital Advertising     21  Digital Advertising   
1593  Data analyst           Child Care     21           Child Care   
1598  Data analyst           LexisNexis      4           LexisNexis   
1600  Data analyst        Esri Software     21        Esri Software   
1606  Data analyst       Zend Framework     10       Zend Framework   

              SBERT_SKILL  %by_job  PARETO  
1                     SQL    53.03    True  
7                  Python     9.18    True  
12       Mac

In [ ]:
#by restricting the roster only to the skills that are reported by at least 1% of the sample, seems to be representative enough of each data profession
filtered_result = result_pareto.loc[result_pareto['%by_job'] >1].drop_duplicates('SBERT_SKILL')
print(filtered_result)

          DATA_TITLE                       SBERT_SKILL_x  SUM_0  \
1       Data analyst                                 SQL   3453   
7       Data analyst                              Python    598   
12      Data analyst                    Machine Learning    384   
16      Data analyst                        Data Science    375   
20      Data analyst                       Data Analysis   2663   
...              ...                                 ...    ...   
5799  Data architect                              Quartz      1   
5803  Data architect                          Windows CE      1   
5819  Data architect          Electronic Medical Records      1   
5820  Data architect  Application Performance Management      1   
5821  Data architect                    Atlassian Bamboo      1   

                                   SKILL                         SBERT_SKILL  \
1                                    SQL                                 SQL   
7                                 P

In [ ]:
result_pareto.loc[result_pareto['%by_job'] >1].drop_duplicates('SBERT_SKILL')

DATA_TITLE  SUM_0                             SKILL  \
1       Data analyst   3453                               SQL   
7       Data analyst    598                            Python   
12      Data analyst    384                  Machine Learning   
16      Data analyst    375                      Data Science   
20      Data analyst   2663                     Data Analysis   
...              ...    ...                               ...   
3376  Data architect      2     Test Driven Development (TDD)   
3436  Data architect      2              Lifecycle Management   
3682  Data architect      2   Network Management Center (NMC)   
3804  Data architect      2  Application Lifecycle Management   
4453  Data architect      2       Continuous Integration (CI)   

                           SBERT_SKILL  %by_job  PARETO  
1                                  SQL     53.0    True  
7                               Python      9.0    True  
12                    Machine Learning      6.0    True  
16                        Data Science      6.0    True  
20                       Data Analysis     41.0    True  
...                                ...      ...     ...  
3376     Test Driven Development (TDD)      3.0   False  
3436              Lifecycle Management      3.0   False  
3682   Network Management Center (NMC)      3.0   False  
3804  Application Lifecycle Management      3.0   False  
4453       Continuous Integration (CI)      3.0   False  

[219 rows x 6 columns]

In [ ]:
filtered_result = result_pareto.loc[(result_pareto['%by_job'] >2) & (result_pareto['PARETO'] == True)].groupby('DATA_TITLE').count()
print(filtered_result)

                SBERT_SKILL_x  SUM_0  SKILL  SBERT_SKILL  %by_job  PARETO
DATA_TITLE                                                               
Data analyst               71     71     71           71       71      71
Data architect             67     67     67           67       67      67
Data engineer              84     84     84           84       84      84
Data scientist             63     63     63           63       63      63


In [ ]:
filtered_result = result_pareto.loc[(result_pareto['%by_job'] >1) & (result_pareto['PARETO'] == True)].drop_duplicates('SBERT_SKILL')

In [ ]:
filtered_result.to_csv('/content/pareto_skills.csv', index=True)
files.download('/content/pareto_skills.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Final

In [ ]:
skills_w_id = pd.read_csv('./skills_w_id.csv', sep = ',')
skill_map = pd.read_csv('./skill_map.csv', sep = ';')
pareto_skills = pd.read_csv('./pareto_skills 5.csv', sep = ';')

In [ ]:
skills_w_id.shape[0]

139229

In [ ]:
skills_w_id.columns

Index(['Unnamed: 0', 'SOCIAL_PROFILE_ID', 'PREDICTED_GENDER_ID', 'RAW_TITLE',
       'SKILL', 'IS_PARSED_FROM_PROFILE_SUMMARY',
       'IS_INFERRED_FROM_EXPERIENCE', 'IS_PARSED_FROM_SKILLS_SECTION',
       'DATA_TITLE'],
      dtype='object')

In [ ]:
col_to_drop = ['Unnamed: 0', 'IS_PARSED_FROM_PROFILE_SUMMARY',
       'IS_INFERRED_FROM_EXPERIENCE', 'IS_PARSED_FROM_SKILLS_SECTION']

skills_w_id = skills_w_id.drop(columns=col_to_drop)

In [ ]:
skills_w_id.columns

Index(['SOCIAL_PROFILE_ID', 'PREDICTED_GENDER_ID', 'RAW_TITLE', 'SKILL',
       'DATA_TITLE'],
      dtype='object')

In [ ]:
skill_map.columns

Index(['Unnamed: 0', 'SKILL_x', 'SBERT_SKILL'], dtype='object')

In [ ]:
col_to_drop = ['Unnamed: 0']

skill_map = skill_map.drop(columns=col_to_drop)

In [ ]:
skill_title_sbert = pd.merge(skills_w_id, skill_map, left_on='SKILL', right_on='SKILL_x', how='left')

In [ ]:
skill_title_sbert.shape[0]

139229

In [ ]:
skill_title_sbert.columns

Index(['SOCIAL_PROFILE_ID', 'PREDICTED_GENDER_ID', 'RAW_TITLE', 'SKILL',
       'DATA_TITLE', 'SKILL_x', 'SBERT_SKILL'],
      dtype='object')

In [ ]:
col_to_drop = ['Unnamed: 0']

pareto_skills = pareto_skills.drop(columns=col_to_drop)
pareto_skills.shape[0]
pareto_skills.columns

Index(['SBERT_SKILL'], dtype='object')

In [ ]:
pareto_skills.shape[0]

193

In [ ]:
pareto_skills.nunique()

SBERT_SKILL    193
dtype: int64

In [ ]:
pareto_list = pareto_skills['SBERT_SKILL'].unique()

In [ ]:
skill_final_red = skill_title_sbert[skill_title_sbert['SBERT_SKILL'].isin(pareto_list)]

In [ ]:
skill_title_sbert.shape[0]

109184

In [ ]:
skill_final_red.columns

Index(['SOCIAL_PROFILE_ID', 'PREDICTED_GENDER_ID', 'RAW_TITLE', 'SKILL',
       'DATA_TITLE', 'SKILL_x', 'SBERT_SKILL'],
      dtype='object')

In [ ]:
col_to_drop = ['SKILL_x']

skill_final_red = skill_final_red.drop(columns=col_to_drop)

In [ ]:
skill_final_red.to_csv('/content/skill_final_red.csv', index=True)
files.download('/content/skill_final_red.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
 skill_final_red['SBERT_SKILL'].nunique()

193

In [ ]:
#END RESULT: By reducing the nr of skills (first by similarity, then by representativity) from 2769 to 193 (-93%) the sample size decreased only marginally, from 138k to 110k (-25%)